In [ ]:
import os
import sys
import json
sys.path.append("../../../")
current_file_dir = os.getcwd()


from prompt_scope.core.llms.dashscope_llm import DashscopeLLM
from prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer import StaticTipsOptimizer

# Step1: Load data

In [ ]:
with open("../../data/college_medical_statistics/sample_train_data.jsonl", "r") as f:
    train_set = []
    for line in f:
        train_set.append(json.loads(line))

with open("../../data/college_medical_statistics/sample_test_data.jsonl", "r") as f:
    test_set = []
    for line in f:
        test_set.append(json.loads(line))

with open("../../data/college_medical_statistics/init_system_prompt.txt", "r") as f:
    init_system_prompt = f.read().strip()

# Step2: Define a scoring function

In [ ]:
def is_good_case(prediction, ground_truth):
    prediction = prediction.split("<answer>")[-1].split("</answer>")[0].strip()
    ground_truth = ground_truth
    if ground_truth in prediction:
        return True, 1 # Good case, score
    else:
        return False, 0 # Bad case, score

# Step3: Run optimizer

In [ ]:
infer_llm = DashscopeLLM(model="qwen-plus", temperature=0.0)
optim_llm = DashscopeLLM(model="qwen-max", temperature=0.0)

In [ ]:
stg = StaticTipsOptimizer(
    init_system_prompt=init_system_prompt,
    infer_llm=infer_llm,
    optim_llm=optim_llm,
    train_set=train_set,
    test_set=test_set,
    is_good_case_func=is_good_case,
    details_save_dir=f"{current_file_dir}/details_result",
    save_steps = 30,
    epoch = 10
)
optimized_prompt = stg.run()